Dear correctors,
we are aware that our program has some problems. First of all - the runtime is astronomic. We suppose the error lies somewhere in our model (i.e. exercise 2), probably in the for-loop. Secondly, the sampling (sess.run()) regularly throws an (Iterator-)OutOfRangeError. We tried to eliminate this one, but when we truncated the training loop (replaced the while loop with a for loop) to shorten the runtime for testing purposes, the Error didn't occur.

In [ ]:
import numpy as np
import tensorflow as tf
import struct
import matplotlib.pyplot as plt
import random

In [ ]:
# 1 The Dataset
# 1.1 read data
text = open("holy_grail.txt",'r').read()

In [ ]:
# 1.2 prepare the dataset
vocabulary = list(set(text))
vocab_len = len(vocabulary)
char_to_int_dict = {v:i for i,v in enumerate(vocabulary)}
int_to_char_dict = {i:v for i,v in enumerate(vocabulary)}
helperint = 0

numberlist = []
for t in text:
    numberlist.append(char_to_int_dict[t])

In [ ]:
# 1.3 construct the dataset
# creating the sequences
sequence_length = 25

sequences= []

for i in range(len(numberlist)-25):
    sequences.append(numberlist[i:(i+25)])

input_sequences = sequences[:-1]
target_sequences = sequences[1:]


# creating the dataset

tf.reset_default_graph()
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))
print(dataset)

batch_size = 1
dataset = dataset.batch(batch_size, drop_remainder=True)
print(dataset)

#iterator

iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)
init_op = iterator.make_initializer(dataset)

In [ ]:
# 2 The Model
# 2.1 input

next_batch = iterator.get_next()
print(next_batch)

inputs = next_batch[0]
targets = next_batch[1]

inputs = tf.cast(inputs, tf.int64)
targets = tf.cast(targets, tf.int64)

inputs_onehot = tf.one_hot(inputs, vocab_len)
targets_onehot = tf.one_hot(targets, vocab_len)

remembered_hidden = tf.placeholder(tf.float32,shape=(100))

In [ ]:
# 2.2 RNN

hidden_states = []
logits_list = []

weights_xh = tf.Variable(tf.random_normal([vocab_len, 100], stddev = 0.1))
weights_hh = tf.Variable(tf.random_normal([100, 100], stddev = 0.1))
weights_hy = tf.Variable(tf.random_normal([100, vocab_len], stddev = 0.1))

biases_h = tf.Variable(tf.zeros([100]))
biases_y = tf.Variable(tf.zeros([vocab_len]))

hidden_state = remembered_hidden

for j in range(batch_size):
    inputs_cur = inputs_onehot[j]
    for i in range(sequence_length):
        # einsum does the same as matmul here but works for tensors with different dimensions
        # (i.e. 1-dim vector and 2-dim matrix)
        hidden_state = tf.nn.tanh(tf.einsum("x,xh->h", inputs_cur[i], weights_xh) + tf.einsum("H,Hh->h",hidden_state, weights_hh) + biases_h)
        logits = tf.einsum("h,hy->y", hidden_state, weights_hy) + biases_y
        hidden_states.append(hidden_state)
        logits_list.append(logits)
    
print("done")

In [ ]:
# 2.3 Output, Loss and Optimizer
output = tf.nn.softmax(logits_list[-1])
hidden_last = hidden_states[-1]

# calculate loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = targets_onehot, logits = logits_list)
loss = tf.reduce_mean(cross_entropy)

# optimizer
optimizer = tf.train.AdamOptimizer()
training_step = optimizer.minimize(loss)

In [ ]:
# 3 Training
# Summary
tf.summary.scalar('loss', loss)

merged_summaries = tf.summary.merge_all()

train_writer = tf.summary.FileWriter('./summaries/train/')

epochs = 10

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    global_step = 0
    
    # epoch loop
    for i in range(epochs):
        hidden_old = [0.0] * 100
        hidden_new = [0.0] * 100
        print("\n\n")
        print("Epoch Nr: " + str(i))
        # training part
        sess.run(init_op, feed_dict={remembered_hidden: hidden_old})
        print("Training")
        while True:
        #for i in range(5):
            #print("In training...")
            try:
                #print("Try")
                #print("Epoch Nr: " + str(i))
                _, summary, hidden_new  = sess.run((training_step, merged_summaries, hidden_last), feed_dict={remembered_hidden: hidden_old})
                # 4 "Validiation"
                train_writer.add_summary(summary, global_step)
                hidden_old = hidden_new
                global_step += 1
            except tf.errors.OutOfRangeError:
                print("Except")
                break
                
        # 4.1 Sampling
        # shouldn't be one of the last input sequences
        print("Sampling")
        sampling_sequence = [random.choice(input_sequences[:-200])]
        
        sampled_letters = []
        for i in range(200):
            #print(sampling_sequence)
            #print("i: " + str(i))
            # sampling sequence as fake target values
            samplingset = tf.data.Dataset.from_tensor_slices((sampling_sequence, sampling_sequence))
            #print(samplingset)
            sampling_batch_size = 1
            samplingset = samplingset.batch(sampling_batch_size)
            iterator = tf.data.Iterator.from_structure(samplingset.output_types,
                                           samplingset.output_shapes)
            init_sampling = iterator.make_initializer(samplingset)
            _, sampled = sess.run((init_sampling, output), feed_dict={remembered_hidden: hidden_old})
            next_letter = np.random.choice(a = vocabulary, p = sampled)
            sampled_letters.append(next_letter)
            sampling_sequence = [sampling_sequence[0][1:]]
            #print(sampling_sequence)
            sampling_sequence[0].append(char_to_int_dict[next_letter])
            #print(next_letter, end = "")
        
        print("".join(sampled_letters))
            
        
print("done")